In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym

2.0.0-alpha0
2.2.4-tf


/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning: Passing (type

In [2]:
import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,Reshape
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import multiprocessing
import yaml
import jieba

In [3]:
class Config(object):
    #数据集路径
    dataSource = "../single.txt"
    stopWordSource = "../stopword.txt"
    #分词后保留大于等于最低词频的词
    miniFreq=1
    #统一输入文本序列的定长，取了所有序列长度的均值。超出将被截断，不足则补0
    sequenceLength = 30 
    batchSize=64
    epochs=50
    numClasses = 6
    #训练集的比例
    rate = 0.8  
    #生成嵌入词向量的维度
    embeddingSize = 200
    #卷积核数
    numFilters = 128
    #卷积核大小
    filterSizes = [1,2,3,4,5]
    dropoutKeepProb = 0.5
    #L2正则系数
    l2RegLambda = 0.01
# 实例化配置参数对象
config = Config()

In [4]:
config.batchSize

64

In [5]:
#中文语料
#设置输出日志
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

file = open("../single.txt") 
sentences=[]
with open('../single.txt') as fr:
    for line in fr.readlines():
        temp=line.strip().split('\t')
#         sentences.append(jieba.lcut(temp[0]))
        sentences.append(list(temp[0]))

model = word2vec.Word2Vec(sentences,size=config.embeddingSize,
                     min_count=config.miniFreq,
                     window=10,
                     workers=multiprocessing.cpu_count(),sg=1,
                     iter=20)
model.save('../word2VecModel')

In [6]:
model = gensim.models.Word2Vec.load('../word2VecModel')
model.wv.vocab.keys()

dict_keys(['我', '要', '了', '解', '不', '是', '单', '身', '哦', '那', '个', '呃', '回', '头', '再', '说', '好', '吗', '没', '有', '啊', '找', '到', '哪', '里', '呀', '就', '贴', '画', '你', '搞', '错', '吧', '行', '现', '在', '外', '面', '办', '事', '情', '寺', '庙', '拉', '出', '来', '朋', '友', '支', '付', '宝', '睡', '觉', '嗯', '用', '谢', '已', '经', '哈', '接', '电', '话', '这', '边', '哎', '投', '多', '少', '年', '前', '的', '大', '姐', '上', '班', '呢', '方', '便', '需', '他', '们', '完', '处', '理', '对', '钱', '想', '法', '段', '时', '间', '些', '先', '进', '什', '么', '但', '转', '女', '士', '意', '思', '退', '打', '哼', '资', '料', '哟', '听', '筒', '明', '白', '改', '忙', '咱', '活', '老', '婆', '试', '很', '久', '之', '发', '晚', '点', '比', '较', '开', '车', '也', '手', '机', '候', '给', '宅', '急', '三', '二', '毛', '线', '例', '小', '心', '会', '闹', '着', '玩', '太', '国', '知', '道', '同', '空', '正', '聊', '像', '陪', '又', '拿', '可', '以', '还', '最', '近', '建', '一', '常', '嘞', '天', '啥', '等', '致', '号', '然', '后', '怎', '善', '交', '象', '帮', '注', '销', '掉', '把', '账', '户', '离', '异', '谈', '早', '脑', '都', '人', '家', '刚', '住', '始', '放',

In [7]:
model['信']

/Library/Python/3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.29857790e-02,  4.91257943e-02, -2.41246507e-01,  4.34472322e-01,
        3.98935467e-01,  1.27987146e-01, -2.76224352e-02, -3.06991041e-01,
        8.95354003e-02,  5.17200269e-02,  1.94119826e-01,  1.30522355e-01,
       -1.27411127e-01, -2.04960108e-01, -2.97124535e-01, -3.92304081e-03,
        3.44849825e-01, -2.29072377e-01, -4.17496860e-01, -1.85405642e-01,
       -9.22740400e-02,  2.06548288e-01, -4.10946384e-02, -2.59862561e-02,
        2.37759486e-01,  1.96844414e-01, -5.80747500e-02, -2.77664274e-01,
       -5.41746952e-02,  1.81302086e-01,  5.53342630e-04,  5.28733552e-01,
       -1.10973738e-01, -8.29479471e-02,  8.15340318e-03,  1.34053767e-01,
        1.32584825e-01,  3.29553276e-01,  6.69638515e-02,  9.62608680e-02,
        3.24829012e-01,  2.89054811e-01, -1.71548724e-02, -7.92213529e-02,
        1.02995627e-01, -6.97705224e-02, -4.08988863e-01,  2.24829257e-01,
       -1.40099123e-01,  8.06888193e-02,  3.97735894e-01, -1.14948623e-01,
       -1.00040007e-02, -

In [8]:
# 数据预处理的类，生成训练集和测试集
class Dataset(object):
    def __init__(self, config):
        self.dataSource = config.dataSource
        self.stopWordSource = config.stopWordSource  
        # 每条输入的序列处理为定长
        self.sequenceLength = config.sequenceLength  
        self.embeddingSize = config.embeddingSize
        self.rate = config.rate
        self.miniFreq=config.miniFreq
        self.stopWordDict = {}
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        self.wordEmbedding =None
        self.n_symbols=0
        self.wordToIndex = {}
        self.indexToWord = {}
        
    def readData(self, filePath):
        text=[]
        label=[]
        with open(filePath) as fr:
            for line in file:
                temp=line.strip().split('\t')
                text.append(temp[0])
                label.append(temp[1])
        print('data:',len(text),len(label))
#         texts = [jieba.cut(document) for document in text]
        texts = [list(document) for document in text]
        return texts, label
    
    def readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    def getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        #中文
        model = gensim.models.Word2Vec.load('../word2VecModel')
        vocab = []
        wordEmbedding = []
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        wordEmbedding.append(np.zeros(self.embeddingSize))
        vocab.append("UNK")
        wordEmbedding.append(np.random.randn(self.embeddingSize))
        for word in words:
            vector =model[word]
            vocab.append(word)
            wordEmbedding.append(vector)           
        return vocab, np.array(wordEmbedding)
    
    def genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        allWords = [word for review in reviews for word in review]
        #去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        #统计词频，排序
        wordCount = Counter(subWords)  
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        #去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= self.miniFreq ]
        #获取词列表和顺序对应的预训练权重矩阵
        vocab, wordEmbedding = self.getWordEmbedding(words)
        
        self.wordEmbedding = wordEmbedding
        
        self.wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self.indexToWord = dict(zip(list(range(len(vocab))), vocab))
        self.n_symbols = len(self.wordToIndex) + 1
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self.wordToIndex, f)
        with open("../wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self.indexToWord, f)
            
    def reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论里面的词，根据词表，映射为index表示
        每条评论 用index组成的定长数组来表示
        """
        review=list(review)
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]
        return reviewVec

    def genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        reviews = []
        labels = []
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self.reviewProcess(x[i], self.sequenceLength, self.wordToIndex)
            reviews.append(reviewVec)
            labels.append([y[i]])    
        trainIndex = int(len(x) * rate)
        
        #trainReviews = sequence.pad_sequences(reviews[:trainIndex], maxlen=self.sequenceLength)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        #evalReviews = sequence.pad_sequences(reviews[trainIndex:], maxlen=self.sequenceLength)
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")
        return trainReviews, trainLabels, evalReviews, evalLabels
         
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        #读取停用词
        self.readStopWord(self.stopWordSource)
        #读取数据集
        reviews, labels = self.readData(self.dataSource)
        #分词、去停用词
        #生成 词汇-索引 映射表和预训练权重矩阵，并保存
        self.genVocabulary(reviews)
        #初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self.genTrainEvalData(reviews, labels, self.rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels

data = Dataset(config)
data.dataGen()

/Library/Python/3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


data: 19359 19359


In [9]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (15487, 30)
train label shape: (15487, 1)
eval data shape: (3872, 30)


In [10]:
def convolution(config):
    sequence_length=config.sequenceLength
    embedding_dimension=config.embeddingSize
    
    inn = Input(shape=(sequence_length, embedding_dimension, 1))
    cnns = []
    filter_sizes=config.filterSizes
    for size in filter_sizes:
        conv = Conv2D(filters=config.numFilters, kernel_size=(size, embedding_dimension),
                            strides=1, padding='valid', activation='relu')(inn)
        pool = MaxPool2D(pool_size=(sequence_length-size+1, 1), padding='valid')(conv)
        cnns.append(pool)
    outt =concatenate(cnns)
    model = Model(inputs=inn, outputs=outt)
    return model

def cnn_mulfilter(n_symbols,embedding_weights,config):
    model =Sequential([
        Embedding(input_dim=n_symbols, output_dim=config.embeddingSize,
                        weights=[embedding_weights],
                        input_length=config.sequenceLength, trainable=True),
        Reshape((config.sequenceLength, config.embeddingSize, 1)),
        convolution(config),
        Flatten(),
#         Dense(10, activation='relu',kernel_regularizer=regularizers.l2(config.l2RegLambda)),
#         Dropout(config.dropoutKeepProb),
        Dense(6, activation='softmax')])
    
#     model.compile(loss='SparseCategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
    model.compile(optimizer=optimizers.Adam(lr=1e-3),
                 loss=losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
    return model
    
wordEmbedding = data.wordEmbedding
n_symbols=data.n_symbols
model = cnn_mulfilter(n_symbols,wordEmbedding,config)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 200)           208000    
_________________________________________________________________
reshape (Reshape)            (None, 30, 200, 1)        0         
_________________________________________________________________
model (Model)                (None, 1, 1, 640)         384640    
_________________________________________________________________
flatten (Flatten)            (None, 640)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3846      
Total params: 596,486
Trainable params: 596,486
Non-trainable params: 0
_________________________________________________________________


In [11]:
x_train = data.trainReviews
y_train = data.trainLabels
x_eval = data.evalReviews
y_eval = data.evalLabels

# y_train = to_categorical(y_train)
# y_eval = to_categorical(y_eval)

wordEmbedding = data.wordEmbedding
n_symbols=data.n_symbols

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=10, mode='auto')

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('./model/best_model/model_{epoch:02d}-{val_accuracy:.2f}.hdf5', save_best_only=True, save_weights_only=True)
# history = model.fit(x_train, y_train, batch_size=config.batchSize, epochs=config.epochs, validation_split=0.2,shuffle=True, callbacks=[reduce_lr,early_stopping,model_checkpoint])
history = model.fit(x_train, y_train, batch_size=config.batchSize, epochs=config.epochs, validation_split=0.2,shuffle=True, callbacks=[model_checkpoint])

#验证
scores = model.evaluate(x_eval, y_eval)

#保存模型
yaml_string = model.to_yaml()
with open('./model/textCNN.yml', 'w') as outfile:
    outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
model.save_weights('./model/textCNN.h5')

print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

Train on 12389 samples, validate on 3098 samples
Epoch 1/50
12389/12389 [==============================] - 7s 556us/sample - loss: 0.9071 - accuracy: 0.6599 - val_loss: 0.7610 - val_accuracy: 0.7198
Epoch 2/50
12389/12389 [==============================] - 6s 516us/sample - loss: 0.7006 - accuracy: 0.7419 - val_loss: 0.7456 - val_accuracy: 0.7256
Epoch 3/50
12389/12389 [==============================] - 6s 518us/sample - loss: 0.6572 - accuracy: 0.7566 - val_loss: 0.7394 - val_accuracy: 0.7331
Epoch 4/50
12389/12389 [==============================] - 6s 524us/sample - loss: 0.6255 - accuracy: 0.7664 - val_loss: 0.7392 - val_accuracy: 0.7292
Epoch 5/50
12389/12389 [==============================] - 7s 526us/sample - loss: 0.6020 - accuracy: 0.7730 - val_loss: 0.7501 - val_accuracy: 0.7243
Epoch 6/50
12389/12389 [==============================] - 7s 526us/sample - loss: 0.5813 - accuracy: 0.7788 - val_loss: 0.7397 - val_accuracy: 0.7318
Epoch 7/50
12389/12389 [===========================

In [12]:
dir(losses)

['BinaryCrossentropy',
 'CategoricalCrossentropy',
 'CategoricalHinge',
 'CosineSimilarity',
 'Hinge',
 'Huber',
 'KLD',
 'KLDivergence',
 'LogCosh',
 'LogLoss',
 'Loss',
 'MAE',
 'MAPE',
 'MSE',
 'MSLE',
 'MeanAbsoluteError',
 'MeanAbsolutePercentageError',
 'MeanSquaredError',
 'MeanSquaredLogarithmicError',
 'Poisson',
 'Reduction',
 'SparseCategoricalCrossentropy',
 'SquaredHinge',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'binary_crossentropy',
 'categorical_crossentropy',
 'categorical_hinge',
 'cosine_similarity',
 'deserialize',
 'get',
 'hinge',
 'kld',
 'kullback_leibler_divergence',
 'logcosh',
 'mae',
 'mape',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_logarithmic_error',
 'mse',
 'msle',
 'poisson',
 'serialize',
 'sparse_categorical_crossentropy',
 'squared_hinge']

In [13]:
y_train

array([[2.],
       [5.],
       [4.],
       ...,
       [5.],
       [2.],
       [2.]], dtype=float32)

In [14]:
to_categorical(y_train)

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [15]:
n_symbols

1040